In [24]:
using Plots
using PyCall
using ControlSystems
#using PyPlot

plotly()

Plots.PlotlyBackend()

In [25]:
#両脚支持期間を導入

zc=0.3
g=9.8
dt=0.001
L=0.15
#歩幅
x=0.05


#支持時間
Tsup=Int32(0.5/dt)
#全脚期間
Tdl=Int32(0.1/dt)

#全体のステップ数
k_max=Int32(1+(Tsup*3)+(Tdl*2))
#目標予見ステップ
M=1000

#支持脚位置
#sx=[0.0 0.0 0.3 0.6 0.9 0.9]
#sy=[0.0 0.2 0.0 0.2 0.0 0.2]

#real support point
real_sx=[0.0; (1/2)*sqrt(3)*L; (1/2)*sqrt(3)*L+x; (1/2)*sqrt(3)*L+x; sqrt(3)*L+x; sqrt(3)*L+2x]
real_sy=[L/2; 0.0; L; L/2; 0.0; L; L/2; 0.0]

#Virtual support point
sx=[(1/2)*(sqrt(3)*L+x) 
    (1/2)*sqrt(3)*L+x 
    (3/4)*sqrt(3)*L+x ]
sy=[L/2 
    3*L/4 
    L/4 ]


#全脚支持期間の目標ZMP生成
#パラメータ決定
x_tdl_a=zeros(Float64,2)
x_tdl_b=zeros(Float64,2)
y_tdl_a=zeros(Float64,2)
y_tdl_b=zeros(Float64,2)

for i in 1:2
    x_tdl_a[i]=(sx[i]-sx[i+1])/(-Tdl*dt)
    x_tdl_b[i]=sx[i]-x_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

for i in 1:2
    y_tdl_a[i]=(sy[i]-sy[i+1])/(-Tdl*dt)
    y_tdl_b[i]=sy[i]-y_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

#目標ZMPベクトル
px_ref=[[sx[1] for s in 1:Tsup]
        [x_tdl_a[1]*s+x_tdl_b[1] for s in 1*Tsup*dt+(1-1)*Tdl*dt:dt:1*Tsup*dt+(1-1)*Tdl*dt+Tdl*dt-dt]
        [sx[2] for s in 1:Tsup]
        [x_tdl_a[2]*s+x_tdl_b[2] for s in 2*Tsup*dt+(2-1)*Tdl*dt:dt:2*Tsup*dt+(2-1)*Tdl*dt+Tdl*dt-dt]
        [sx[3] for s in 1:Tsup+1+M]]
        

py_ref=[[sy[1] for s in 1:Tsup]
    [y_tdl_a[1]*s+y_tdl_b[1] for s in 1*Tsup*dt+(1-1)*Tdl*dt:dt:1*Tsup*dt+(1-1)*Tdl*dt+Tdl*dt-dt]
        [sy[2] for s in 1:Tsup]
    [y_tdl_a[2]*s+y_tdl_b[2] for s in 2*Tsup*dt+(2-1)*Tdl*dt:dt:2*Tsup*dt+(2-1)*Tdl*dt+Tdl*dt-dt]
        [sy[3] for s in 1:Tsup+1+M]]
   

#計算上のZMPベクトル
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max) 

 px_ref

2701-element Array{Float64,1}:
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 0.154904
 ⋮       
 0.244856
 0.244856
 0.244856
 0.244856
 0.244856
 0.244856
 0.244856
 0.244856
 0.244856
 0.244856
 0.244856
 0.244856

In [26]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[1000 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#エラーシステム

#内部状態ベクトル
x=[ sx[1]+0.001
    0.0
    0.0]

y=[ L/2
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0]

4-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0

In [27]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #plotするために記録
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #plotするために記録
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 


xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y


dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR


old_x=x
old_y=y

xu=0
yu=0

P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 0.0

In [28]:
for i in 1:k_max-1  
    #求めた入力を用いてk+1の状態を計算する
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]
    
    k+=1
    
    x_plot_log[k+1]=x[1] #plotするために記録
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #plotするために記録
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    
    dx=x-old_x
    dy=y-old_y
    
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    
    
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y
    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    
    old_x=x
    old_y=y
    
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end
    

In [45]:
plot(t,px_ref,lab="目標ZMP",linewidth=3,linecolor="blue")
plot!(t,x_plot_log,lab="重心位置",linewidth=3,linecolor="red")
plot!(t,px,lab="ZMP",linewidth=3,linecolor="black")
plot!(xlab="時間[ms]",ylab="zmp_x[m]")

In [44]:
plot(t,py_ref,lab="目標ZMP",linewidth=3,linecolor="blue")
plot!(t,y_plot_log,lab="重心位置",linewidth=3,linecolor="red")
plot!(t,py,lab="ZMP",linewidth=3,linecolor="black")
plot!(xlab="時間[ms]",ylab="zmp_y[m]")

In [31]:
plot(t,xv_plot_log,lab="速度")
plot!(xlab="時間[ms]",ylab="速度[m/s]")

In [32]:
plot(t,xa_plot_log,lab="速度")
plot!(xlab="時間[ms]",ylab="加速度[m/s]")

In [37]:
plot(t,yv_plot_log,lab="速度")
plot!(xlab="時間[ms]",ylab="速度[m/s]")

In [38]:
plot(t,ya_plot_log,lab="加速度")
plot!(xlab="時間[ms]",ylab="加速度[m/s]")